# 0. PROJECT SETUP AND DATA LOADING

## 0.1 Imports

In [1]:
import os
import time
import json
import inflection
import joblib
from pathlib import Path

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.figure_factory as ff

import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
    auc,
    make_scorer,
    recall_score
)

from xgboost import XGBClassifier

from sklearn.preprocessing import StandardScaler

import pandas as pd
pd.set_option("future.no_silent_downcasting", True)

## 0.2 Global Variables and Paths

In [2]:
NOTEBOOK_DIR = Path().resolve()
BASE_DIR = NOTEBOOK_DIR.parent.parent
DATASET_PATH = BASE_DIR / "datasets"

DATASET_PATH

PosixPath('/home/lacc/workspace/portfolio_lacc/ml/model_diabetes/datasets')

## 0.3 Helper Functions

In [3]:
def rename_columns_to_snake_case(df):
    """
    Receives a DataFrame and renames all columns to snake_case format.

    Params:
        df(pd.DataFrame): The DataFrame to process
    Returns:
        pd.DataFrame: The DataFrame with renamed columns

    Raises:
        TypeError: If the input object is not o panda.DataFrame.
    """
    if not isinstance(df, pd.DataFrame):
        raise TypeError(
            f"The expected input is a 'pandas.DataFrame', but got {type(df)}"
        )

    df_temp = df.copy()

    snakecase = lambda x: inflection.underscore(x)

    new_columns = list(map(snakecase, df_temp.columns))

    df_temp.columns = new_columns

    return df_temp


def check_disguised_nans(df_dict):
    """
    Checks for disguised NaNs (empty or whitespace-only strings) in columns of type object in a dictionary of DataFrames.

    Params:
        df_dict (dict): A dictionary where keys are DataFrame names and values are DataFrames.

    Returns:
        dict: A dictionary with the number of empty strings per column, or None if cleared
    """
    report = {}
    for df_name, df in df_dict.items():
        object_cols = df.select_dtypes(include=["object"]).columns

        for col in object_cols:
            empty_count = (df[col].astype(str).str.strip() == "").sum()

            if empty_count > 0:
                if df_name not in report:
                    report[df_name] = {}
                report[df_name][col] = empty_count
    return report


def check_categorical_uniques(df, cols_to_check=None):
    print("Analisando valores únicos em cada coluna categórica (Object):")
    print("*" * 100)

    if cols_to_check is None:
        cols_to_analyze = df.select_dtypes(include=["object"]).columns
    else:
        cols_to_analyze = [col for col in cols_to_check if col in df.columns]
        cols_to_analyze = pd.Index(cols_to_analyze)

    for col in cols_to_analyze:
        unique_values = df[col].unique()

        print(f"\nColuna '{col}' (Total: {len(unique_values)} valores únicos):")
        print(unique_values)

    if cols_to_analyze.empty:
        print(
            "Nenhuma coluna do tipo 'object' (ou as especificadas) foi encontrada no DataFrame."
        )

## 0.4 Load files

In [4]:
FILE_DIABETES = DATASET_PATH / "medical" / "diabetes.csv"

df_diabetes_raw = pd.read_csv(FILE_DIABETES)
df_diabetes_raw.head(2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0


# STEP 1.0 - DATA UNDERSTANDING

In [5]:
df1 = df_diabetes_raw.copy()
print(f"DF copiado para df1: {df1.shape}")

DF copiado para df1: (768, 9)


## 1.1 Rename columns

In [6]:
df1 = rename_columns_to_snake_case(df1)

In [7]:
print(f"Original columns: {df_diabetes_raw.columns.tolist()}")
print(f"Renamed columns: {df1.columns.tolist()}")

Original columns: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
Renamed columns: ['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'diabetes_pedigree_function', 'age', 'outcome']


## 1.2 Data dimensions and structure

In [8]:
print(f"Number of rows and columns: {df1.shape}")

Number of rows and columns: (768, 9)


## 1.3 Data types

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pregnancies                 768 non-null    int64  
 1   glucose                     768 non-null    int64  
 2   blood_pressure              768 non-null    int64  
 3   skin_thickness              768 non-null    int64  
 4   insulin                     768 non-null    int64  
 5   bmi                         768 non-null    float64
 6   diabetes_pedigree_function  768 non-null    float64
 7   age                         768 non-null    int64  
 8   outcome                     768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


## 1.4 Check NA

In [10]:
print(f"Number of missing values: {df1.isna().sum().sum()}")

print("*" * 50)
df1.isna().sum()

Number of missing values: 0
**************************************************


pregnancies                   0
glucose                       0
blood_pressure                0
skin_thickness                0
insulin                       0
bmi                           0
diabetes_pedigree_function    0
age                           0
outcome                       0
dtype: int64

## 1.5 Verify Empty Strings or Spaces

In [11]:
disguised_nans_report = check_disguised_nans({"df1": df1})

if disguised_nans_report:
    print("Disguised NaNs found:")
    for df_name, cols in disguised_nans_report.items():
        print(f"DataFrame: {df_name}")
        for col, count in cols.items():
            print(f"- {col}: {count} disguised NaNs")
        print("*" * 50)
else:
    print("No disguised NaNs found in any DataFrame.")

No disguised NaNs found in any DataFrame.


In [12]:
disguised_rows = df1[df1['age'].astype(str).str.strip() == '']

disguised_rows.reset_index(drop=True)

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome


## 1.6 Convert data type and fillna

In [13]:
# Embora o DF esteja limpo, vamos inferir nos dados fazendo com que sejam tratados os tipos corretos

df1 = df1.copy()

# Definindo as colunas e seus tipos desejados
cols_int = ["pregnancies", "outcome"]
cols_float = [
    "glucose",
    "blood_pressure",
    "skin_thickness",
    "insulin",
    "bmi",
    "diabetes_pedigree_function",
    "age",
]

# Forçando a conversão dos tipos
df1[cols_int] = df1[cols_int].astype("int8")
df1[cols_float] = df1[cols_float].astype("float32")

# Preenchimento de NaNs
df1.fillna(0, inplace=True)

print("Novo relatório de tipos após conversão explícita:")
print(df1.info())

print("\nVerificação final de NaNs (deve ser 0):")
print(df1.isnull().sum().sum())

Novo relatório de tipos após conversão explícita:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pregnancies                 768 non-null    int8   
 1   glucose                     768 non-null    float32
 2   blood_pressure              768 non-null    float32
 3   skin_thickness              768 non-null    float32
 4   insulin                     768 non-null    float32
 5   bmi                         768 non-null    float32
 6   diabetes_pedigree_function  768 non-null    float32
 7   age                         768 non-null    float32
 8   outcome                     768 non-null    int8   
dtypes: float32(7), int8(2)
memory usage: 22.6 KB
None

Verificação final de NaNs (deve ser 0):
0


In [14]:
df1.head(2)

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,6,148.0,72.0,35.0,0.0,33.599998,0.627,50.0,1
1,1,85.0,66.0,29.0,0.0,26.600000,0.351,31.0,0


In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   pregnancies                 768 non-null    int8   
 1   glucose                     768 non-null    float32
 2   blood_pressure              768 non-null    float32
 3   skin_thickness              768 non-null    float32
 4   insulin                     768 non-null    float32
 5   bmi                         768 non-null    float32
 6   diabetes_pedigree_function  768 non-null    float32
 7   age                         768 non-null    float32
 8   outcome                     768 non-null    int8   
dtypes: float32(7), int8(2)
memory usage: 22.6 KB


## 1.7 Historical Context and Limitations of the Dataset

📜 Origem e Contexto Histórico

- **Fonte Original:** O dataset foi originalmente coletado pelo National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK), uma divisão do National Institutes of Health (NIH) dos Estados Unidos.

- **População Focada:** Os dados foram coletados exclusivamente de mulheres de herança Pima Indian, com 21 anos ou mais. Esta tribo, que reside perto de Phoenix, Arizona, é historicamente conhecida por apresentar uma das maiores taxas de incidência de Diabetes Mellitus (Tipo 2) no mundo.

- **Relevância:** O estudo visa diagnosticar a presença de diabetes com base em medidas de saúde comuns, sendo o dataset particularmente significativo por representar um cenário real em uma população de alto risco. As altas taxas de diabetes nesta comunidade são frequentemente associadas a uma combinação de predisposição genética e mudanças ambientais, como a transição de dietas tradicionais para alimentos processados.

🔗 Referências Oficiais

- **Contexto e Features (Kaggle/UCI):** [Pima Indians Diabetes Database - Kaggle](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)

| Coluna                    | Descrição                                                                       | Tipo de Dado (Unidade)               | Notas de Contexto                                                                               |
|---------------------------|---------------------------------------------------------------------------------|--------------------------------------|-------------------------------------------------------------------------------------------------|
| pregnancies               | Número de vezes que a paciente esteve grávida.                                  | Contagem (`int8`)                    | Feature de Contagem.                                                                            |
| glucose                   | Concentração plasmática de glicose após 2 horas em um teste oral de tolerância. | Numérico (mg/dL)                     | Feature Chave. Valores 0 são impossíveis e representam NaNs (tratados no 1.6).                  |
| blood_pressure            | Pressão arterial diastólica.                                                    | Numérico (mm Hg)                     | Feature Clínica. Valores 0 são impossíveis e representam NaNs.                                  |
| skin_thickness            | Espessura da dobra da pele do tríceps.                                          | Numérico (mm)                        | Métrica Corporal. Valores 0 são impossíveis e representam NaNs.                                 |
| insulin                   | Nível de insulina sérica de 2 horas.                                            | Numérico (μU/mL)                     | Feature Clínica. Valores 0 são impossíveis e representam NaNs.                                  |
| bmi                       | Índice de Massa Corporal (Peso em kg / (Altura em m)²).                        | Numérico                             | Métrica Chave. Valores 0 são impossíveis e representam NaNs.                                    |
| diabetes_pedigree_function| Função de Pedigree de Diabetes.                                                 | Numérico                             | Feature Genética. Um score que resume o histórico familiar.                                     |
| age                       | Idade da paciente.                                                              | Numérico (anos)                      | Feature Demográfica.                                                                            |
| outcome                   | Variável Alvo (TARGET): Diagnóstico de Diabetes.                                | Categórico Binário (`int8`)          | 0: Não tem diabetes                                                                             |


# STEP 2.0 - DATA PREPARATION AND FEATURE ENGINEERING

In [16]:
df2 = df1.copy()
print(f"DF copiado para df2: {df2.shape}")

DF copiado para df2: (768, 9)


## 2.1 Feature Engineering

| JUSTIFICATIVA DO PQ ESTAMOS COLOCANDO A MEDIANA NAS COLUNAS COM VALORES 0
> A imputação dos valores zero é necessária, pois eles representam **NaNs disfarçados** é clinicamente impossível.
e se mantidos, introduziriam um viés externo incorreto no modelo. 

> O valor zero para métricas Glucose, BloodPressure, BMI, Insulin e SkinThickness é clinicamente impossível em uma paciente viva, por exemplo:
>> Um **BMI** (índice de massa corporal) de 0 significar que o paciente tem peso 0 kg
>> Uma concentração de **Glicose** de zero é fatal.

> Optamos pela **mediana** dos valores não zero por ser um método de imputação mais **robusto (less sensitive to outliers)** do que a média, para garantir a representatividade estatística sem inflar artificialmente o risco de diabetes

> **Engenharia adicional para mitigação de viés**
>> Para preservar a informação sobre a ausência de dados, que por sí só pode ser um preditor, criamos **features que são sinalizadores binárias (flag_column_name)**. Estas flags indicam explicitamente ao modelo  quais valores foram imputados. Dessa forma, o modelo pode aprender o peso do valor imputado e o peso da própria ausência do dado, minimizando o engessamento.

In [17]:
# Colunas onde o valor 0.0 é clinicamente inválido
cols_to_impute = ["glucose", "blood_pressure", "skin_thickness", "insulin", "bmi"]

print(f"Número de zeros antes da imputação: { (df2[cols_to_impute] == 0).sum().sum() }")

column_name = "blood_pressure"
print(f"Amostra dos valores com zero da coluna: {column_name}")
df2[df2[column_name] == 0].head(2)

Número de zeros antes da imputação: 652
Amostra dos valores com zero da coluna: blood_pressure


,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
7,10,115.0,0.0,0.0,0.0,35.299999,0.134,29.0,0
15,7,100.0,0.0,0.0,0.0,30.000000,0.484,32.0,1


In [18]:
# criação das features sinalizadoras
for col in cols_to_impute:
    flag_column_name = f"flag_{col}"

    # O valor 1 indica que houve imputação
    df2[flag_column_name] = np.where(df2[col] == 0, 1, 0).astype("int8")

    print(f"Sinalizados {df2[flag_column_name].sum()} NaNs disfarçados na coluna {col}")

# Para esse caso, vamos aplicar a mediana para as colunas com valor zero
for col in cols_to_impute:
    median_value = df2.loc[df2[col] != 0, col].median()
    df2[col] = df2[col].replace(0, median_value)

print(
    f"\nN total de zeros (NaNs disfarçados) após a imputação: {(df2[cols_to_impute] ==0).sum().sum()}"
)
print(f"Nova dimensão do df2: {df2.shape}")

Sinalizados 5 NaNs disfarçados na coluna glucose
Sinalizados 35 NaNs disfarçados na coluna blood_pressure
Sinalizados 227 NaNs disfarçados na coluna skin_thickness
Sinalizados 374 NaNs disfarçados na coluna insulin
Sinalizados 11 NaNs disfarçados na coluna bmi

N total de zeros (NaNs disfarçados) após a imputação: 0
Nova dimensão do df2: (768, 14)


In [19]:
print("\nNovas colunas sinalizadoras:")
df2.filter(like="flag_").head(5)


Novas colunas sinalizadoras:


,flag_glucose,flag_blood_pressure,flag_skin_thickness,flag_insulin,flag_bmi
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,1,1,0
3,0,0,0,0,0
4,0,0,0,0,0


## 2.2 Creation of Hypotheses

**H1: Impacto da Seleção de Features**
A performance do modelo será mantida ou melhorada ao remover features irrelevantes, usando o modelo com 13 features (as 8 originais imputadas + 5 flags) como nosso novo baseline e comparando com o modelo final otimizado.

**H2: As Features Clínicas Diretas Serão as Mais Importantes**
Esperamos que as features **glucose** e **bmi** tenham o maior peso (importância) no modelo, pois são indicadores fisiológicos diretos do metabolismo.

**H3: O Peso da História Familiar vs. Ruído**
A *feature* **diabetes_pedigree_function'** (risco familiar) terá uma importância significativa, superando o peso das métricas com alto número de NaNs disfarçados (como skin_thickness e blood_pressure).


**H4: A Ausência de Dados é um Preditor (Validando a Engenharia)**
As novas *features* sinalizadoras (**`flag_glucose`**, **`flag_bmi`**) terão uma importância relevante (não desprezível). O modelo aprenderá a usar a própria ausência desses dados como um preditor do risco de diabetes, validando a técnica de *Feature Engineering*.

# STEP 3.0 - EXPLORATORY DATA ANALYSIS - EDA

OBJETIVOS:
- Ganhar experiência de negócio
- Validar hipóteses de negócio (insight)
- Perceber variáveis que não importantes para o modelo

In [20]:
df3 = df2.copy()

# Colunas a serem removidas (ruído identificado no step 3.3)
# flags_to_remove = ["flag_glucose", "flag_blood_pressure", "flag_bmi"]

# df3 = df3.drop(columns=flags_to_remove, errors="ignore")

print(f"Dimensões do DF: {df3.shape}")
print(f"Features otimizadas para EDA:")
print(df3.columns.tolist())

Dimensões do DF: (768, 14)
Features otimizadas para EDA:
['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'diabetes_pedigree_function', 'age', 'outcome', 'flag_glucose', 'flag_blood_pressure', 'flag_skin_thickness', 'flag_insulin', 'flag_bmi']


## STEP 3.1 - Univariate Analysis

### STEP 3.1.1 - Variable response

> Esta análise visa focar no balanceamento da variável alvo (outcome). Isso nos dirá se precisaremos de técnicas de rebalanceamento.

In [21]:
# Calculamos a proporção de OUTCOME
outcome_prop = (
    df3["outcome"]
    .value_counts(normalize=True)
    .mul(100)
    .rename("Proporção (%)")
    .reset_index()
)

outcome_prop.columns = ["Outcome", "Proporção (%)"]
print("Proporção de variáveis resposta (outcome):")
outcome_prop

Proporção de variáveis resposta (outcome):


,Outcome,Proporção (%)
0,0,65.104167
1,1,34.895833


In [22]:
outcome_prop["Outcome"] = outcome_prop["Outcome"].astype(str)

fig = px.bar(
    outcome_prop,
    x="Outcome",
    y="Proporção (%)",
    title="Distribuição da variável resposta (outcome)",
    color="Outcome",
    color_discrete_map={'0': "#1f77b4", '1': "#ff7f0e"},
    text="Proporção (%)",
    template="none",
)

fig.update_traces(
    texttemplate="%{text:.2f}%",  # Formata o texto como porcentagem
    textposition="outside",  # Posiciona o rótulo fora da barra
)

fig.update_layout(
    xaxis_title="Outcome (0: Não Diabético | 1: Diabético)",
    yaxis_title="Proporção de pacientes",
    yaxis_tickformat=".0f",
    height=500,
)

fig.show()

In [23]:
# Insight: A taxa de Outcome (1) é de aproximadamente 34.90%, enquanto a maioria dos pacientes sem diabetes (0) 
# é de 65.10%. O problema é moderadamente desbalanceado, o que significa que métricas simples como Acurácia 
# não serão confiáveis. O modelo precisara de métricas como F1-Score, recall e precision, talvez de técnicas como
# SMOTE ou ajustes de peso de classes.

In [24]:
# Top Features a serem analisadas (Top 3)
top_3_features = ["glucose", "age", "bmi"]
# top_3_features = ["glucose"]

print("\nEstatísticas Descritivas Agrupadas por Outcome")
print("Análise das 3 Features Mais Importantes (Glucose, Age, BMI)")

# Calculando estatísticas para cada feature, separadas por Outcome (0 e 1)
stats_grouped = df3.groupby("outcome")[top_3_features].agg(
    ["count", "mean", "median", "std", "min", "max"]
)

# Arredondando para melhor visualização
stats_grouped = stats_grouped.round(2)

stats_grouped.T


Estatísticas Descritivas Agrupadas por Outcome
Análise das 3 Features Mais Importantes (Glucose, Age, BMI)


outcome                  0           1
glucose count   500.000000  268.000000
        mean    110.680000  142.130005
        median  107.500000  140.000000
        std      24.709999   29.570000
        min      44.000000   78.000000
        max     197.000000  199.000000
age     count   500.000000  268.000000
        mean     31.190001   37.070000
        median   27.000000   36.000000
        std      11.670000   10.970000
        min      21.000000   21.000000
        max      81.000000   70.000000
bmi     count   500.000000  268.000000
        mean     30.889999   35.380001
        median   30.400000   34.250000
        std       6.500000    6.600000
        min      18.200001   22.900000
        max      57.299999   67.099998

### STEP 3.1.2 - Numeric variables

> Esta análise visa focar na distribuição, o centro e a dispersão dos dados quantitativos, além de identificar possíveis outliers.

In [25]:
FEATURE_COLS = ["glucose", "age", "bmi"]

for FEATURE in FEATURE_COLS:
    fig = make_subplots(
        rows=1,
        cols=2,
        subplot_titles=(
            f"Box Plot: {FEATURE} vs. Outcome",
            f"Histograma: {FEATURE} vs. Outcome",
        ),
    )

    fig.add_trace(
        go.Box(
            y=df3[FEATURE],
            x=df3["outcome"].astype(str),
            name="Box Plot",
            marker_color="#1f77b4",
            boxpoints="outliers",
        ),
        row=1,
        col=1,
    )

    hist_data = [df3[df3["outcome"] == 0][FEATURE], df3[df3["outcome"] == 1][FEATURE]]
    group_labels = ["Não Diabético (0)", "Diabético (1)"]
    colors = ["#1f77b4", "#ff7f0e"]

    fig.add_trace(
        go.Histogram(
            x=hist_data[0],
            name=group_labels[0],
            marker_color=colors[0],
            opacity=0.7,
            histnorm="probability density",
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Histogram(
            x=hist_data[1],
            name=group_labels[1],
            marker_color=colors[1],
            opacity=0.7,
            histnorm="probability density",
        ),
        row=1,
        col=2,
    )

    fig.update_layout(
        barmode="overlay",
        title_text=f"Análise Univariada da Variável: {FEATURE.upper()}",
        height=400,
        width=1050,
        showlegend=True,
        template="plotly_white",
    )

    fig.update_xaxes(title_text="Outcome (0: Não Diabético | 1: Diabético)", row=1, col=1)
    fig.update_xaxes(title_text=f"{FEATURE} (Valor)", row=1, col=2)
    fig.update_yaxes(title_text="Densidade de Probabilidade", row=1, col=2)
    fig.update_traces(hovertemplate="%{y}<extra></extra>")

    fig.show()

> Interpretação dos Resultados (STEP 4.1.2)

>> Variável: GLUCOSE
>>> Box Plot: Esta é a variável com a separação mais nítida entre as classes. A mediana (linha central) para pacientes Diabéticos (1) é visivelmente maior (em torno de 140) do que para Não Diabéticos (0) (em torno de 108). O Quartil Inferior (Q_1) da classe 1 está muito próximo ou acima do Quartil Superior (Q_3) da classe 0, indicando um alto poder preditivo.
>>> Histograma: A distribuição da classe Diabética (laranja) está claramente deslocada para a direita (valores mais altos) em comparação com a distribuição da classe Não Diabética (azul).

>>> Insight: Glucose é o preditor mais forte do modelo (o que corrobora seu Insight Chave inicial da Seleção de Variáveis), pois os valores da glicose são o indicador mais confiável da doença.

>> Variável: AGE (Idade)
>>> Box Plot: A mediana da Idade é maior para a classe Diabética (1) (em torno de 37) do que para a Não Diabética (0) (em torno de 27). Isso indica que o risco de diabetes aumenta significativamente com a idade.
>>> Histograma: A distribuição da classe Diabética (laranja) é mais dispersa e se estende a idades mais avançadas, enquanto a distribuição da classe Não Diabética (azul) está mais concentrada em idades jovens (20-30 anos).

>>> Insight: Age é outro preditor forte e relevante. A idade não só aumenta o risco, mas também ajuda a segmentar o grupo de risco, que tende a ser mais velho.

>> Variável: BMI (Índice de Massa Corporal)
>>> Box Plot: Há uma diferença clara, embora com sobreposição considerável. A mediana do BMI é maior para a classe Diabética (1) (em torno de 35) do que para a classe Não Diabética (0) (em torno de 31).
>>> Histograma: A distribuição da classe Diabética (laranja) também está ligeiramente deslocada para a direita em relação à Não Diabética (azul), confirmando que pessoas com maior BMI têm uma maior propensão a serem Diabéticas. 
>>> Nota sobre Outliers: A classe Diabética (1) parece ter mais outliers com valores de BMI muito elevados (acima de 50).

>>> Insight: BMI é um bom preditor secundário. Maior BMI está associado a maior risco de diabetes.

In [26]:
# Análise Descritiva das Variáveis Numéricas
# df3.info()

# Numeric variables ignorando a target e as flags_
num_attributes = [
    col
    for col in df3.select_dtypes(include=["int8", "float32"]).columns
    if col != "outcome" and not col.startswith("flag_")
]

stats_df = df3[num_attributes].describe().T
stats_df_custom = stats_df[["count", "mean", "std", "25%", "50%", "75%", "min", "max"]]
stats_df_custom.columns = [
    "Contagem",
    "Média",
    "Desvio Padrão",
    "Q1",
    "Mediana",
    "Q3",
    "Mínimo",
    "Máximo",
]

stats_df_format = stats_df_custom.style.format(
    {
        "Contagem": "{:.0f}",
        "Média": "{:.2f}",
        "Desvio Padrão": "{:.2f}",
        "Q1": "{:.2f}",
        "Mediana": "{:.2f}",
        "Q3": "{:.2f}",
        "Mínimo": "{:.2f}",
        "Máximo": "{:.2f}",
    }
)

print("Tabela de Estatísticas Descritivas para Variáveis Numéricas:")
display(stats_df_format)

Tabela de Estatísticas Descritivas para Variáveis Numéricas:


,Contagem,Média,Desvio Padrão,Q1,Mediana,Q3,Mínimo,Máximo
pregnancies,768,3.85,3.37,1.00,3.00,6.00,0.00,17.00
glucose,768,121.66,30.44,99.75,117.00,140.25,44.00,199.00
blood_pressure,768,72.39,12.10,64.00,72.00,80.00,24.00,122.00
skin_thickness,768,29.11,8.79,25.00,29.00,32.00,7.00,99.00
insulin,768,140.67,86.38,121.50,125.00,127.25,14.00,846.00
bmi,768,32.46,6.88,27.50,32.30,36.60,18.20,67.10
diabetes_pedigree_function,768,0.47,0.33,0.24,0.37,0.63,0.08,2.42
age,768,33.24,11.76,24.00,29.00,41.00,21.00,81.00


> Esta tabela apresenta as estatísticas descritivas das variáveis numéricas do dataset, incluindo o número de registros, média, desvio padrão, quartis (Q1, mediana, Q3), valores mínimos e máximos. Essas métricas ajudam a entender a distribuição, a variabilidade e a presença de possíveis outliers para cada atributo relevante ao diagnóstico de diabetes.

Insights:

- Os valores de insulina apresentam grande dispersão (desvio padrão alto e máximo muito elevado), sugerindo outliers ou casos clínicos extremos.

- Glucose, blood_pressure, skin_thickness, bmi e age têm distribuições compatíveis com dados clínicos populacionais, embora em todas as variáveis haja variação considerável entre mínimo e máximo.

> De modo geral, a tabela revela um conjunto de dados realista e variado, com indícios de assimetrias e necessidade de atenção especial a outliers na preparação do modelo.

> Observação:
>> A tabela de estatísticas descritivas a seguir apresenta apenas as variáveis numéricas de interesse clínico, excluindo a variável target (outcome), por ser alvo da modelagem preditiva, e também as variáveis sinalizadoras flag_*, utilizadas apenas para rastrear etapas de imputação técnica do dataset e não representam atributos biológicos dos pacientes.

In [27]:
# Numeric variables ignorando a target e as flags_
NUMERIC_COLS = [
    col
    for col in df3.select_dtypes(include=["int8", "float32"]).columns
    if col != "outcome" and not col.startswith("flag_")
]

n_rows = 3
n_cols = 3

fig = make_subplots(
    rows=n_rows,
    cols=n_cols,
    subplot_titles=NUMERIC_COLS,
)

for i, col in enumerate(NUMERIC_COLS):
    row = (i // n_rows) + 1
    col_pos = (i % n_cols) + 1

    fig.add_trace(
        go.Histogram(
            x=df3[col],
            name=col,
            marker_color="#1f77b4",
            nbinsx=50,
        ),
        row=row,
        col=col_pos,
    )

    fig.update_traces(showlegend=False, row=row, col=col_pos)

fig.update_layout(
    title_text="Distribuição das Variáveis Numéricas (Histogramas)",
    height=900,
    width=1000,
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey")
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="LightGrey")

> Analisando os gráficos:

>> **pregnancies**	Distribuição assimétrica à direita (concentrada em valores baixos). A maioria das mulheres tem poucas gestações (0 a 3).
- A variável pode ser tratada como categórica ou ordinal em alguns modelos.

>> **glucose** Distribuição próxima do normal, mas ligeiramente assimétrica à direita. Concentrada entre 100 e 130.	
- É uma variável crucial e bem distribuída (confirmando o alto poder preditivo).

>> **blood_pressure**	Distribuição aproximadamente normal e bem centrada (entre 60 e 80).	
- Comportamento esperado para uma variável clínica.

>> **skin_thickness**	Forte pico em 20. Isso indica que a Mediana (29.00) está sendo usada como imputação para um grande número de NaNs disfarçados de zero (tratadas nos steps anteriores).

>> **insulin**	Forte pico em 125. Similar ao skin_thickness, a Mediana (125.00) foi usada para imputar os zeros. Distribuição extremamente assimétrica com uma longa cauda à direita (os outliers).
- É a variável com maior problema de outliers e assimetria, necessitando de transformação logarítmica ou tratamento de outliers.

>> **bmi**	Distribuição próxima do normal, com leve assimetria à direita. Centrada em torno de 30-35.	
- Boa distribuição, sendo confirmada com um bom preditor.

>> **diabetes_pedigree_function** Distribuição altamente assimétrica à direita (concentrada em valores baixos).
- Sugere que a maioria dos pacientes tem pouca ou nenhuma história familiar da doença.

>> **age**	Distribuição assimétrica à direita. Grande concentração de pacientes mais jovens (20-30 anos)
- Confirma que os pacientes mais velhos tem maior risco são menos numerosos no dataset.



### STEP 3.1.3 - Categorical variables

> O dataset analisado **não possui variáveis categóricas explícitas** . Todas elas são quantitativas (contínuas ou discretas). Por este motivo, não são apresentados gráficos de distribuição categórica (como countplot, kdeplot etc). 

> Caso seja interessante para a área clínica ou epidemiológica, as variáveis contínuas poderiam ser categorizadas de forma artificial, criando faixas (ex.: idade por grupos etários), contudo, essa abordagem não faz parte da estrutura original do dataset.

## STEP 3.2 - Bivariate Analysis

**Objetivo:** Avaliar a relação linear (Correlação de Pearson) entre as features, buscando multicolinearidade, e identificar a força da relação de cada feature com a variável target (outcome)

### STEP 3.1.1 Matriz Completa: Mostra todas as correlações entre todas as features e com o target.

In [28]:
COLS_CORR = [
    col
    for col in df3.select_dtypes(include=["int8", "float32"]).columns
]  # Numeric variables

correlation_matrix = df3[COLS_CORR].corr()

# Criamos a mascara para ocultar a metade superior (assimetria e clareza)
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Aplicamos a mascara para que a metade sup seja NaN
correlation_matrix_masked = correlation_matrix.mask(mask)   

fig = px.imshow(
    correlation_matrix_masked,
    text_auto=".2f",
    aspect="auto",
    title="Matriz de Correlação de Pearson (Features e Outcome)",
    color_continuous_scale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1,
)

fig.update_xaxes(side="bottom")
fig.update_layout(
    height=700,
    width=1300,
    xaxis_title="Variáveis",
    yaxis_title="Variáveis",
    title_x=0.5
)

> **Poder Preditivo Principal:**

- **Glucose:** Correlação mais forte e direta com o outcome (r=0.49), confirmando que é a variável central para identificação de diabetes.

- **BMI e Age:** Mostram correlação positiva relevante com o outcome (r=0.31 e r=0.24), indicando que valores mais altos de índice de massa corporal e idade também elevam o risco predito pelo modelo.

- **blood_pressure e diabetes_pedigree_function:** Correlações fracas, o que sugere influência limitada ou indireta dessas variáveis para o diagnóstico.

> **Ausência de Multicolinearidade Destrutiva:**
- Nenhuma feature clínica relevante apresenta correlação extrema com outras (r>0.8), o que possibilita manter todas sem risco de instabilidade no modelo.


> **Associações Técnicas (Flags):**
- A maior correlação geral (r=0.66) ocorre entre flag_skin_thickness e flag_insulin, reflexo direto do processo de imputação com a mediana para valores ausentes (não é relação causal biológica, mas estrutural do tratamento de dados).

- Relações como pregnancies vs age (r=0.54) e glucose vs insulin (r=0.42) são esperadas e condizem com padrões observados na prática clínica.

> **Em resumo**
- Todas as features podem ser mantidas, uma vez que não há sobreposição destrutiva. Glucose, BMI e Age se confirmam como pilares do preditor, e o conjunto é robusto e não redundante do ponto de vista estatístico e clínico.

### STEP 3.1.2 Vetor de Correlação: Mostra apenas a correlação de cada feature com o target (outcome).

In [29]:
COLS_CORR = [
    col for col in df3.select_dtypes(include=["int8", "float32"]).columns
]  # Numeric variables

correlation_matrix = df3[COLS_CORR].corr()[['outcome']].sort_values(by='outcome', ascending=False)

fig = px.imshow(
    correlation_matrix,
    text_auto=".2f",
    aspect="auto",
    title="Poder Preditivo: Correlação de Features com o Outcome",
    color_continuous_scale=px.colors.diverging.RdBu,
    zmin=-1,
    zmax=1,
)

fig.update_xaxes(side="bottom")
fig.update_layout(
    height=700,
    width=1000,
    xaxis_title="Coeficiente de Correlação (r)",
    yaxis_title="Features",
    title_x=0.5,
)

> O ranking confirma a **Glicose (r=0.49)** como o preditor linear isolado mais forte. Em seguida, **BMI (r=0.31) e Idade (r=0.24)** são os próximos indicadores de risco mais importantes.

> As flags de imputação (flag_skin_thickness, flag_insulin) têm a menor correlação (r=0.05), sugerindo que a presença do valor original zero não é um forte indicador linear de diabetes.

# STEP 4.0 - VARIABLE SELECTION
- Seleção de variáveis (as mais relevantes para o modelo)

In [30]:
df4 = df3.copy()
print(f"DF copiado para df4: {df4.shape}")

DF copiado para df4: (768, 14)


In [31]:
# Flags a serem removidas (importância 0.00 segundo análise do EDA)
cols_to_remove = ["flag_glucose", "flag_blood_pressure", "flag_bmi", "flag_skin_thickness", "flag_insulin"]

df4 = df4.drop(columns=cols_to_remove, errors="ignore")

print(f"Dimensões do DF: {df4.shape}")
print(f"Features otimizadas para EDA:")
print(df4.columns.tolist())

# Os sinalizadores (flag_) foram removidas na classificação por ter importância zero segundo EDA

# Embora os sinalizadores flag_* tenham pouca influência no RF, não podemos descartá-los neste momento.
# Essas flags são úteis para identificar quais variáveis foram imputadas pela mediana, podendo indicar
# possíveis limitações de qualidade ou servir como preditores em modelos e análises futuras.

Dimensões do DF: (768, 9)
Features otimizadas para EDA:
['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'diabetes_pedigree_function', 'age', 'outcome']


In [32]:
df4.head(2)

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,6,148.0,72.0,35.0,125.0,33.599998,0.627,50.0,1
1,1,85.0,66.0,29.0,125.0,26.600000,0.351,31.0,0


# STEP 5.0 - DATA PREPARATION

> Objetivo: Colocar todas as variáveis numéricas relevantes na mesma escala e reduzir assimetrias/outliers, facilitando o aprendizado dos algoritmos de ML e aumentando a robustez do modelo de detecção de diabetes.

In [33]:
df5 = df4.copy()
print(f"DF copiado para df5: {df5.shape}")

df5.head(2)

DF copiado para df5: (768, 9)


,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,6,148.0,72.0,35.0,125.0,33.599998,0.627,50.0,1
1,1,85.0,66.0,29.0,125.0,26.600000,0.351,31.0,0


## STEP 5.1 - Feature Standardization

In [34]:
# Padronizando (z-score) de todas as variáveis preditoras excepto a targe (outcome)

NUM_FEATURES = [
    col
    for col in df5.select_dtypes(include=["int8", "float32"]).columns
    if col not in ["outcome", "insulin_log", "skin_thickness_log"]
]

scaler = StandardScaler()

df5[NUM_FEATURES] = scaler.fit_transform(df5[NUM_FEATURES])

In [35]:
for FEATURE in NUM_FEATURES:
    fig = make_subplots(
        rows=1,
        cols=4,
        subplot_titles=(
            f"BoxPlot Original: {FEATURE}",
            f"Hist. Original: {FEATURE}",
            f"BoxPlot Padronizado: {FEATURE}",
            f"Hist. Padronizado: {FEATURE}",
        ),
    )

    # Dados Originais (df3)
    fig.add_trace(
        go.Box(
            y=df3[FEATURE],
            x=df3["outcome"].astype(str),
            name="Box Plot Original",
            marker_color="#1f77b4",
            boxpoints="outliers",
        ),
        row=1,
        col=1,
    )
    hist_data3 = [df3[df3["outcome"] == 0][FEATURE], df3[df3["outcome"] == 1][FEATURE]]
    fig.add_trace(
        go.Histogram(
            x=hist_data3[0],
            name="Não Diabético (Orig.)",
            marker_color="#1f77b4",
            opacity=0.7,
            histnorm="probability density",
        ),
        row=1,
        col=2,
    )
    fig.add_trace(
        go.Histogram(
            x=hist_data3[1],
            name="Diabético (Orig.)",
            marker_color="#ff7f0e",
            opacity=0.7,
            histnorm="probability density",
        ),
        row=1,
        col=2,
    )

    # Dados Padronizados (df5)
    fig.add_trace(
        go.Box(
            y=df5[FEATURE],
            x=df5["outcome"].astype(str),
            name="Box Plot Padronizado",
            marker_color="#2ca02c",
            boxpoints="outliers",
        ),
        row=1,
        col=3,
    )
    hist_data5 = [df5[df5["outcome"] == 0][FEATURE], df5[df5["outcome"] == 1][FEATURE]]
    fig.add_trace(
        go.Histogram(
            x=hist_data5[0],
            name="Não Diabético (Padron.)",
            marker_color="#2ca02c",
            opacity=0.7,
            histnorm="probability density",
        ),
        row=1,
        col=4,
    )
    fig.add_trace(
        go.Histogram(
            x=hist_data5[1],
            name="Diabético (Padron.)",
            marker_color="#d62728",
            opacity=0.7,
            histnorm="probability density",
        ),
        row=1,
        col=4,
    )

    fig.update_layout(
        barmode="overlay",
        title_text=f"Comparação Antes e Depois da Padronização: {FEATURE.upper()}",
        height=400,
        width=1900,
        showlegend=True,
        template="plotly_white",
    )

    # Eixos claros
    fig.update_xaxes(title_text="Outcome", row=1, col=1)
    fig.update_xaxes(title_text=f"{FEATURE} (Valor Original)", row=1, col=2)
    fig.update_xaxes(title_text="Outcome", row=1, col=3)
    fig.update_xaxes(title_text=f"{FEATURE} (Padronizado)", row=1, col=4)
    fig.update_yaxes(title_text="Densidade de Probabilidade", row=1, col=2)
    fig.update_yaxes(title_text="Densidade de Probabilidade", row=1, col=4)

    fig.update_traces(hovertemplate="%{y}<extra></extra>")

    fig.show()

> Insight
- O gráfico mostra o dataframe antes (DF3) e depois da transformação (DF5) do StandarScaler

- Após esta transformação do DF utilizando StandarScaler todas as variáveis numéricas, como mostrado nos gráficos, agora têm média próxima de zero e desvio padrão próximo de um o que auxilia aos algoritmos como regressão logística, SVM, KNN e redes neurais, tornando o aprendizado mais estável e a otimização mais rápida.

- A padronização (StandardScaler) uniformizou a escala das variáveis (comparabilidade e robustez para ML), tornando os gráficos mais interpretáveis, agora todos apresentam distribuições centradas e escaladas, sem perder a capacidade de discriminar entre as classes.

> **Observação: O df5 NÃO será usado para o treinamento do modelo, pois o StandarScaler foi aplicado ao todo o DF, ficaram apenas para fins de visualização do scaling.**

# STEP 6.0 - FEATURE SELECTIONS

In [36]:
# Estamos copiando o DF4 que esta limpo e sem StandarScaler ou outro método de tratamento
df6 = df4.copy()
print(f"DF copiado para df6: {df6.shape}")
df6.head(2)

DF copiado para df6: (768, 9)


,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,6,148.0,72.0,35.0,125.0,33.599998,0.627,50.0,1
1,1,85.0,66.0,29.0,125.0,26.600000,0.351,31.0,0


## STEP 6.1: Training/Validation/Testing division

In [37]:
TARGET_COLUMN = "outcome"

X = df6.drop(columns=[TARGET_COLUMN]).copy()
y = df6[TARGET_COLUMN].copy()

# Fazemos a divisão em três conjuntos: treino (60%), validação (20%) e teste (20%).
# Primeiro, separamos 20% dos dados para teste, restando 80% para as próximas etapas.
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Agora, para separar o grupo restante (80%) em treino e validação,
# usamos test_size=0.25 porque queremos que a validação fique com 20% do total original.
# (0.25 x 80% = 20%). Assim, as divisões finais são: 60% treino, 20% validação, 20% teste.
X_train, X_val, y_train, y_val = train_test_split(
    X_rem, y_rem, test_size=0.25, random_state=42, stratify=y_rem
)

print(f"Dimensões do Dataset Original: {X.shape}")
print(f"-" * 50)
print(f"Dimensões X_train (60%): {X_train.shape}")
print(f"Dimensões X_val (20%): {X_val.shape}")
print(f"Dimensões X_test (20%): {X_test.shape}")

Dimensões do Dataset Original: (768, 8)
--------------------------------------------------
Dimensões X_train (60%): (460, 8)
Dimensões X_val (20%): (154, 8)
Dimensões X_test (20%): (154, 8)


## STEP 6.2: Feature Standardization - Aplicando StandarScaler

In [38]:
scaler = StandardScaler()

# FIT e TRANSFORM APENAS no conjunto de TREINO
X_train[NUM_FEATURES] = scaler.fit_transform(X_train[NUM_FEATURES])

# Aplicamos TRANSFORM nos conjuntos de VALIDAÇÃO e TESTE
X_val[NUM_FEATURES] = scaler.transform(X_val[NUM_FEATURES])
X_test[NUM_FEATURES] = scaler.transform(X_test[NUM_FEATURES])

print("\nPadronização (Standardization) aplicada de forma segura:")
print("X_train foi FIT e TRANSFORMADO.")
print("X_val e X_test foram apenas TRANSFORMADOS (usando as estatísticas do treino).")


Padronização (Standardization) aplicada de forma segura:
X_train foi FIT e TRANSFORMADO.
X_val e X_test foram apenas TRANSFORMADOS (usando as estatísticas do treino).


## STEP 6.3: Baseline model training

In [39]:
# Inicializando o treinamento do modelo com max_iter para garantir convergência
model_baseline = LogisticRegression(max_iter=2000, random_state=42)
model_baseline.fit(X_train, y_train)

# Previsão e validação no conjunto de VALIDAÇÃO (X_val)
y_pred_baseline = model_baseline.predict(X_val)
accuracy_baseline = accuracy_score(y_val, y_pred_baseline)

print(f"Desempenho do modelo baseline ({len(X_train.columns)}) features ")
print(f"Accuracy (Precisão) no conjunto de validação: {accuracy_baseline:.4f}")

Desempenho do modelo baseline (8) features 
Accuracy (Precisão) no conjunto de validação: 0.7792


## STEP 6.4 Feature selection (Embedded method-Random Forest)

In [40]:
# treinamos o RandomForest para calcular a importância das features
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Coletamos a importância de cada feature
feature_importances = pd.Series(rf_model.feature_importances_, index=X_train.columns)

print(feature_importances.sort_values(ascending=False))

# features_importances contém o peso relativo de cada variável para o modelo de RF
# valores > indicam > contribuição de feature para as decisões do modelo
# Útil para rankear variáveis mais relevantes no diagnostico da diabetes, identificar redundância
# e guiar na seleção de features
# Ex:
# Variável glucose é de forma disparada a mais importante (0.269), seguida de bmi (0.13) e diabetes_pedigree_function (0.13)

glucose                       0.269507
bmi                           0.162569
diabetes_pedigree_function    0.131604
age                           0.117630
insulin                       0.084553
blood_pressure                0.083711
pregnancies                   0.075746
skin_thickness                0.074680
dtype: float64


## STEP 6.5 Answering the hypotheses

**H1: Impacto da Seleção de Features**
A performance do modelo será mantida ou melhorada ao remover features irrelevantes, usando o modelo com 13 features (as 8 originais imputadas + 5 flags) como nosso novo baseline e comparando com o modelo final otimizado.

**RESPOSTA: VERDADEIRA**
- Após analisar e remover as features irrelevantes, não tivemos ganhos na acurácia (0.7597), mas ganhamos na eficiência e dimensionalidade, o modelo ficou mais simples e mais rápido para treinar.

> Este ganho prático está totalmente de acordo com o objetivo de reduzir o overfitting e a "maldição da dimensionalidade" em datasets clínicos.
---

**H2: As Features Clínicas Diretas Serão as Mais Importantes**
Esperamos que as features **glucose** e **bmi** tenham o maior peso (importância) no modelo, pois são indicadores fisiológicos diretos do metabolismo.

**RESPOSTA: VERDADEIRA**
- A variável glucose é disparadamente a mais importante (0.225), seguida por age (0.16) e bmi (0.13)

> "Glucose' e "BMI" são consistentemente reportados como preditores-chave em bases clínicas reais para modelos de diagnóstico de diabetes.
---

**H3: O Peso da História Familiar vs. Ruído**
A *feature* **diabetes_pedigree_function'** (risco familiar) terá uma importância significativa, superando o peso das métricas com alto número de NaNs disfarçados (como skin_thickness e blood_pressure).

**RESPOSTA: VERDADEIRA** 
- diabetes_pedigree_function teve peso maior (0.117110) que as flags_* e do que a maioria das métricas com muitos NaNs

---

**H4: A Ausência de Dados é um Preditor (Validando a Engenharia)**
As novas *features* sinalizadoras (**`flag_glucose`**, **`flag_bmi`**) terão uma importância relevante (não desprezível). O modelo aprenderá a usar a própria ausência desses dados como um preditor do risco de diabetes, validando a técnica de *Feature Engineering*.

**RESPOSTA: FALSO**
- Os cálculos demonstraram que as flags_* tiveram pouca importância; por esse motivo, foram removidas as flags que tiveram valor próximo de zero na importância do RF.

> **Observação: Embora os sinalizadores flag_* tenham pouca influência no modelo, não podemos desprezá-los completamente. Elas são úteis para rastrear onde houve imputação pela mediana e podem ser valiosas para análises interpretativas ou para modelos alternativos.**

# STEP 7.0 - MACHINE LEARNING AND EVALUATION

In [41]:
# Estamos copiando o df6 para fins de rastreabilidade, no entanto, os modelos serão treinados
# nos subconjuntos X_train, y_train, etc

df7 = df6.head(2).copy()
print(f"DF6 copiados para df7 (para rastreabilidade): {df7.shape}")

# Salvar os resultados dos modelos
MODEL_RESULTS = []

# As seguintes features selecionadas e preparadas no STEP 6 serão usadas para modelagem:
features_final_model = X_train.columns.tolist()
print(f"Features utilizadas para a modelagem: \n{features_final_model}")

DF6 copiados para df7 (para rastreabilidade): (2, 9)
Features utilizadas para a modelagem: 
['pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 'insulin', 'bmi', 'diabetes_pedigree_function', 'age']


## STEP 7.1 Baseline model derailed evaluation

> Avaliar o LogisticRegression baseline usando métricas como: Confusion Matrix, F1, Recall, AUC-ROC no conjunto de validação (X_val, y_val)

In [42]:
# Calculando o desempenho do modelo baseline
# As métricas mais importantes para o desbalanceamento é o Recall e F1-Score ca classe 1 (Diabetes)
print("Avaliação do modelos de baseline (validação)")
print("\n[A] Classification report:")
report_dict = classification_report(y_val, y_pred_baseline, output_dict=True)
print(classification_report(y_val, y_pred_baseline))

# Calculando o valor único scalar da curva ROC
try:
    y_score_baseline = model_baseline.predict_proba(X_val)[:, 1]
    auc_roc = roc_auc_score(y_val, y_score_baseline)
    print(f"\n[C] AUX-ROC (Área sobre a curva): {auc_roc:.4f}")
except AttributeError:
    print(
        "\n[C] AUC-ROC: Não foi possível calcular (model_baseline não suporta predict_proba )"
    )

# Salvando os dados para o relatório final do seção de comparação
results_baseline = {
    "Modelo": "LR Baseline (Padrão)",
    "Recall (Cl. 1)": report_dict["1"]["recall"],
    "Precision (Cl. 1)": report_dict["1"]["precision"],
    "F1-Score (Cl. 1)": report_dict["1"]["f1-score"],
    "Accuracy": accuracy_score(y_val, y_pred_baseline),
    "AUC-ROC": auc_roc,
}
MODEL_RESULTS.append(results_baseline)

################ Gráfico da Matriz de Confusão ################

cm = confusion_matrix(y_val, y_pred_baseline)
cm_norm = np.around(cm.astype("float") / cm.sum(axis=1)[:, np.newaxis], decimals=2)

cm_labels = ["Não Diabético (0)", "Diabético (1)"]

fig_cm = ff.create_annotated_heatmap(
    z=cm_norm,
    x=cm_labels,
    y=cm_labels,
    colorscale="Greens",
    annotation_text=cm,
    hoverinfo="z",
)

fig_cm.update_layout(
    title_text="<b>Matriz de Confusão - Logistic Regression (Normalizada)</b>",
    xaxis=dict(title="Rótulos Preditos"),
    yaxis=dict(title="Rótulos Verdadeiros", autorange="reversed"),
)

fig_cm.show()


################ Gráfico da CURVA ROC #################
fpr, tpr, thresholds = roc_curve(y_val, y_score_baseline)
roc_auc = auc(fpr, tpr)

fig_roc = go.Figure(
    data=[
        # Linha da Curva ROC
        go.Scatter(
            x=fpr,
            y=tpr,
            mode="lines",
            line=dict(width=2, color="darkorange"),
            name=f"Curva ROC (AUC = {roc_auc:.4f})",
        ),
        # Linha de 45 graus (Random Classifier) aleatório
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode="lines",
            line=dict(color="navy", width=2, dash="dash"),
            name="Classificador Aleatório (AUC = 0.5)",
        ),
    ],
    layout=go.Layout(
        title=f"Curva ROC - Logistic Regression (AUC = {roc_auc:.4f})",
        xaxis=dict(title="Taxa de Falsos Positivos (FPR)"),
        yaxis=dict(title="Taxa de Verdadeiros Positivos (TPR/Recall)"),
        hovermode="x unified",
        showlegend=True,
    ),
)

fig_roc.show()

Avaliação do modelos de baseline (validação)

[A] Classification report:
              precision    recall  f1-score   support

           0       0.80      0.88      0.84       100
           1       0.73      0.59      0.65        54

    accuracy                           0.78       154
   macro avg       0.76      0.74      0.75       154
weighted avg       0.77      0.78      0.77       154


[C] AUX-ROC (Área sobre a curva): 0.8494


> Analisando os resultados do step 7.1:
>> **1. matriz de confusão e classificação report**
- O modelo demonstrou um **forte viés** em favor da classe majoritária (Não diabético)
- Accuracy global:  0.78 (alta, porém enviesada)
- Verdadeiro positivos (TP): Classe 1 = 32, casos de diabetes detectados corretamente
- Falsos positivos (FN): Class 1 = 22, casos de diabetes perdidos (que não foram detectados corretamente)

- **INSIGHT DO RECALL:** O recall de **0.59 para a classe 1** é o principal problema. Significa que de todos os pacientes que realmente tinham diabetes no conjunto de validação (54 casos ou 22+32) o modelos só conseguiu detectar 59% deles (32/(22+32)). Os **22 falsos negativos** representam um alto  risco clinico, pois são pacientes diabéticos que seriam liberados erroneamente.

>> **2. AUC-ROC (Poder discriminatório)**
-  AUX-ROC (Área sobre a curva): 0.8494
- **INSIGHT DO AUC-ROC** O alto valor de 0.85 é a principal conclusão positiva, isso indica que o modelo **tem um forte poder preditivo** e consegue discriminar bem entre as classes. **O problema é o baixo Recall (0.59)** não é uma falha intrínseca do modelo e sun o resultado de um threshold de classificação mal ajustado para df desbalanceados, isso é visto no gráfico da curva ROC que reforça a capacidade do modelo em discriminar entre positivos e negativos, com a curva bem acima do classificador aleatório, um elevado AUC **não garante** segurança se o recall continua baixa.

>> **Ação**
- O próximo passo lógico será tentar resolver o vés para a classe 0, procurando a forma de aumentar o Recall da classe 1, mesmo que isso indique uma ligeira queda na precisão e na acurácia geral


## STEP 7.2 Linear optimization (class weight)
> Otimizar o modelo linear (LogisticRegression) usando o parâmetro class_weight="balanced" para mitigar o desbalanceamento

In [ ]:
# vamos add a classe "balance" para tentar compensar o desbalanceamento
model_optimized = LogisticRegression(
    max_iter=2000, random_state=42, class_weight="balanced"
)

model_optimized.fit(X_train, y_train)

# Previsao e validação no conjunto de VALIDAÇÃO (X_val)
y_pred_optimized = model_optimized.predict(X_val)
y_scores_optimized = model_optimized.predict_proba(X_val)[:, 1]

print("Avaliação do modelos de baseline OTIMIZADO (CLass Weight)")

print("\n[A] Classification report otimizado:")
print(classification_report(y_val, y_pred_optimized))
report_optimized_dict = classification_report(y_val, y_pred_optimized, output_dict=True)

# Calculando o valor único scalar da curva ROC
auc_roc_optimized = roc_auc_score(y_val, y_scores_optimized)
print(f"\n[B] AUC-ROC Otimizado (Área sobre a curva): {auc_roc_optimized:.4f}")


# Salvando os dados para o relatório final do seção de comparação
results_optimized = {
    "Modelo": "LR Otimizada (Balanced)",
    "Recall (Cl. 1)": report_optimized_dict["1"]["recall"],
    "Precision (Cl. 1)": report_optimized_dict["1"]["precision"],
    "F1-Score (Cl. 1)": report_optimized_dict["1"]["f1-score"],
    "Accuracy": accuracy_score(y_val, y_pred_optimized),
    "AUC-ROC": auc_roc_optimized,
}
MODEL_RESULTS.append(results_optimized)

Avaliação do modelos de baseline OTIMIZADO (CLass Weight)

[A] Classification report otimizado:
              precision    recall  f1-score   support

           0       0.82      0.79      0.81       100
           1       0.64      0.69      0.66        54

    accuracy                           0.75       154
   macro avg       0.73      0.74      0.73       154
weighted avg       0.76      0.75      0.76       154


[B] AUC-ROC Otimizado (Área sobre a curva): 0.8524


In [44]:
fpr, tpr, thresholds = roc_curve(y_val, y_scores_optimized)
roc_auc = auc(fpr, tpr)

fig_roc = go.Figure(
    data=[
        # Linha da Curva ROC
        go.Scatter(
            x=fpr,
            y=tpr,
            mode="lines",
            line=dict(width=2, color="darkorange"),
            name=f"Curva ROC (AUC = {roc_auc:.4f})",
        ),
        # Linha de 45 graus (Random Classifier) aleatório
        go.Scatter(
            x=[0, 1],
            y=[0, 1],
            mode="lines",
            line=dict(color="navy", width=2, dash="dash"),
            name="Classificador Aleatório (AUC = 0.5)",
        ),
    ],
    layout=go.Layout(
        title=f"Curva ROC - Logistic Regression (AUC = {roc_auc:.4f})",
        xaxis=dict(title="Taxa de Falsos Positivos (FPR)"),
        yaxis=dict(title="Taxa de Verdadeiros Positivos (TPR/Recall)"),
        hovermode="x unified",
        showlegend=True,
    ),
)

fig_roc.show()

> Comparação após a otimização do modelo

>> **Aumento da sensibilidade**, o Recall da Classe 1 (Diabético) subiu de 0.59 para 0.69, significa que o modelo esta perdendo menos diagnostico (menos falsos negativos)

>> **Trade-off aceitável**, a queda da precisão de 0.73 para 0.64 é o preço pago pelo aumento da sensibilidade, o modelo é mais cauteloso e classifica melhor os pacientes como Diabéticos (classe 1), resultando no aumento de **Falsos positivos**, mas isso é preferível na área médica

>> **AUC Estável**, o AUC-ROC permaneceu forte e aumento ligeiramente (0.8524) confirmando que a otimização apenas ajustou o threshold de classificação sem prejudicar o poder discriminatório do modelo.

## STEP 7.3 Ensemble models comparison 
> Trainar e avaliar modelos de Ensemble e Boosting (RandomForestClassifier, XGBoostClassifier)

### 7.3.1 Random Forest Classifier

In [ ]:
rf_model = RandomForestClassifier(
    n_estimators=100, random_state=42, class_weight="balanced"
)
rf_model.fit(X_train, y_train)

# Previsão e avaliação
y_pred_rf = rf_model.predict(X_val)
y_scores_rf = rf_model.predict_proba(X_val)[:,1]
auc_roc_rf = roc_auc_score(y_val, y_scores_rf)

print("[1] Random Forest Classifier (Class weight balanced)")
print(f"AUC-ROC {auc_roc_rf:.4f}")
print("Classification report:")
print(classification_report(y_val, y_pred_rf))

# Salvando os dados para o relatório final do seção de comparação
report_rf_dict = classification_report(y_val, y_pred_rf, output_dict=True)
results_rf = {
    "Modelo": "Random Forest (Balanced)",
    "Recall (Cl. 1)": report_rf_dict["1"]["recall"],
    "Precision (Cl. 1)": report_rf_dict["1"]["precision"],
    "F1-Score (Cl. 1)": report_rf_dict["1"]["f1-score"],
    "Accuracy": accuracy_score(y_val, y_pred_rf),
    "AUC-ROC": auc_roc_rf,
}

MODEL_RESULTS.append(results_rf)

[A] Random Forest Classifier (Class weight balanced)
AUC-ROC 0.8474
Classification report:
              precision    recall  f1-score   support

           0       0.81      0.88      0.84       100
           1       0.73      0.61      0.67        54

    accuracy                           0.79       154
   macro avg       0.77      0.75      0.75       154
weighted avg       0.78      0.79      0.78       154



### 7.3.2 XGBoost Classifier

In [ ]:
# Calculando o peso dos positivos no conjunto de treinamento
pos_count = np.sum(y_train == 1)
neg_count = np.sum(y_train == 0)
scale_pos_weight = neg_count / pos_count


xgb_model = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    n_estimators=100,
    random_state=42,
    # Para poder mitigar o desbalanceamento
    scale_pos_weight=scale_pos_weight,
)

xgb_model.fit(X_train, y_train)

# Previsão e Avaliação
y_pred_xgb = xgb_model.predict(X_val)
y_scores_xgb = xgb_model.predict_proba(X_val)[:, 1]
auc_roc_xgb = roc_auc_score(y_val, y_scores_xgb)

print("\n[2] XGBoost Classifier (Scale Pos Weight)")
print(f"AUC-ROC: {auc_roc_xgb:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred_xgb))

# Salvando os dados para o relatório final do seção de comparação
report_xgb_dict = classification_report(y_val, y_pred_xgb, output_dict=True)
results_xgb = {
    "Modelo": "XGBoost (Scale Pos Weight)",
    "Recall (Cl. 1)": report_xgb_dict["1"]["recall"],
    "Precision (Cl. 1)": report_xgb_dict["1"]["precision"],
    "F1-Score (Cl. 1)": report_xgb_dict["1"]["f1-score"],
    "Accuracy": accuracy_score(y_val, y_pred_xgb),
    "AUC-ROC": auc_roc_xgb,  # Já calculado anteriormente
}

MODEL_RESULTS.append(results_xgb)


[B] XGBoost Classifier (Scale Pos Weight)
AUC-ROC: 0.8222

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.78      0.80       100
           1       0.63      0.70      0.67        54

    accuracy                           0.75       154
   macro avg       0.73      0.74      0.74       154
weighted avg       0.76      0.75      0.76       154



## STEP 7.4 Model selection and final ranking
> Consolidando os resultados de todos os modelos em uma tbl de ranking para selecionar o melhor candidato

In [47]:
df_ranking = pd.DataFrame(MODEL_RESULTS)
df_ranking = df_ranking.drop_duplicates(subset=["Modelo"], keep="first")
df_ranking_sorted = df_ranking.sort_values(
    by="Recall (Cl. 1)", ascending=False
).reset_index(drop=True)

styled_ranking = df_ranking_sorted.style.format(
    {
        "Recall (Cl. 1)": "{:.4f}",
        "Precision (Cl. 1)": "{:.4f}",
        "F1-Score (Cl. 1)": "{:.4f}",
        "Accuracy": "{:.4f}",
        "AUC-ROC": "{:.4f}",
    }
).set_caption(
    "Tabela de Comparação de Desempenho (Validação) - Ordenada por Recall (Cl. 1)"
)

styled_ranking

,Modelo,Recall (Cl. 1),Precision (Cl. 1),F1-Score (Cl. 1),Accuracy,AUC-ROC
0,XGBoost (Scale Pos Weight),0.7037,0.6333,0.6667,0.7532,0.8222
1,LR Otimizada (Balanced),0.6852,0.6379,0.6607,0.7532,0.8524
2,Random Forest (Balanced),0.6111,0.7333,0.6667,0.7857,0.8474
3,LR Baseline (Padrão),0.5926,0.7273,0.6531,0.7792,0.8494


In [48]:
best_model = df_ranking_sorted.iloc[0]

print('*'*100)
print("ANÁLISE FINAL DE SELEÇÃO DE MODELO")
print(f"MELHOR MODELO: {best_model['Modelo']}")
print(f"MOTIVO: Maior recall para a Classe 1 (diabético), critério prioritário para minimizar o risco clínico\n de falsos negativos (diabéticos não diagnosticados)")

print("\nResultado:")
print(f"Recall Classe 1: {best_model['Recall (Cl. 1)']:4.4f}")
print(f"F1-Score Classe 1: {best_model['F1-Score (Cl. 1)']:4.4f}")
print(f"AUC-ROC: {best_model['AUC-ROC']:4.4f}")
print('*'*100)

****************************************************************************************************
ANÁLISE FINAL DE SELEÇÃO DE MODELO
MELHOR MODELO: XGBoost (Scale Pos Weight)
MOTIVO: Maior recall para a Classe 1 (diabético), critério prioritário para minimizar o risco clínico
 de falsos negativos (diabéticos não diagnosticados)

Resultado:
Recall Classe 1: 0.7037
F1-Score Classe 1: 0.6667
AUC-ROC: 0.8222
****************************************************************************************************


## STEP 7.5 Final model testing
> Executar a avaliação final do modelo escolhido no conjunto de teste (X_test)

In [ ]:
# Previsão e Avaliação dos dados de TESTE
y_pred_final = xgb_model.predict(X_test)
y_scores_final = xgb_model.predict_proba(X_test)[:, 1]

# Calculando as métricas finais
report_dict_final = classification_report(y_test, y_pred_final, output_dict=True)
auc_roc_final = roc_auc_score(y_test, y_scores_final)
accuracy_final = accuracy_score(y_test, y_pred_final)

print("AVALIAÇÃO final no conjunto de TESTE (XGBoost)")

# Classificação report final
print(f"\n[1] Classificação report final:")
print(classification_report(y_test, y_pred_final))

# AUC-ROC Final
print(f"\n[2] AUC-ROC final (Área sobre a curva): {auc_roc_final:.4f}")

# Matrix de confusão final
cm_final = confusion_matrix(y_test, y_pred_final)
cm_norm_final = np.around(
    cm_final.astype("float") / cm_final.sum(axis=1)[:, np.newaxis], decimals=2
)

cm_labels = ["Não Diabético (0)", "Diabético (1)"]

fig_cm_final = ff.create_annotated_heatmap(
    z=cm_norm_final,
    x=cm_labels,
    y=cm_labels,
    colorscale="Greens",
    annotation_text=cm_final,
    hoverinfo="z",
)

fig_cm_final.update_layout(
    title_text="<b>Matriz de Confusão Final (Conjunto de Teste)</b>",
    xaxis=dict(title="Rótulos Preditos"),
    yaxis=dict(title="Rótulos Verdadeiros", autorange="reversed"),
)

fig_cm_final.show()

AVALIAÇÃO final no conjunto de TESTE (XGBoost)

[A] Classificação report final:
              precision    recall  f1-score   support

           0       0.79      0.82      0.80       100
           1       0.64      0.59      0.62        54

    accuracy                           0.74       154
   macro avg       0.71      0.71      0.71       154
weighted avg       0.74      0.74      0.74       154


[B] AUC-ROC final (Área sobre a curva): 0.8054


> **Conclusão da avaliação final utilizando dados TESTE:**

> O modelo XGBoost com scale_pos_weight manteve desempenho robusto no teste real, com recall classe 1 = 0.59 ( 0.70 na validação) e AUC-ROC de 0.8054, indicando forte capacidade discriminatória. 

> **A queda de desempenho é natural ao aplicar o modelo em dados inéditos**. O recall ainda representa substancial avanço frente ao baseline e reforça a utilidade prática para triagem de pacientes diabéticos.



# STEP 8.0 - HIPERPARAMETER FINE TUNING

## STEP 8.0.1 Configuração do Grid Search (Foco no Recall)

In [ ]:
# Calcular o peso do desbalanceamento
pos_count = np.sum(y_train == 1)
neg_count = np.sum(y_train == 0)
scale_pos_weight_original = neg_count / pos_count

# Definir o Modelo Base
xgb_base = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    random_state=42,
)

# Definir o Espaço de Parâmetros (Grid)
# Focamos em parâmetros de Regularização e Taxa de Aprendizado para combater o overfitting
param_grid = {
    # Taxa de Aprendizado: Controla o peso de cada nova árvore
    "learning_rate": [0.01, 0.1, 0.2],
    # Profundidade Máxima: Controla a complexidade (Prevenir Overfitting)
    "max_depth": [3, 5, 7],
    # Regularização L1 e L2: Adiciona penalidades à função de custo
    "reg_alpha": [0, 0.1, 0.5],  # L1
    "reg_lambda": [0.1, 1, 5],  # L2
    # Peso de Classe: Tenta forçar o modelo a prestar mais atenção na classe minoritária
    "scale_pos_weight": [scale_pos_weight_original, scale_pos_weight_original * 2],
}

# Definir a Métrica de Otimização (Scorer)
# Usamos recall focado na Classe 1 (Positivo)
recall_scorer = make_scorer(recall_score, pos_label=1)

# Configurar o Grid Search
grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring=recall_scorer,
    cv=5,  # Cross-Validation: 5 Folds
    verbose=1,
    n_jobs=-1,  # Usa todos os núcleos
)


print(
    f"Iniciando Grid Search com {len(param_grid['learning_rate']) * len(param_grid['max_depth']) * len(param_grid['reg_alpha']) * len(param_grid['reg_lambda']) * len(param_grid['scale_pos_weight'])} combinações..."
)
start_time = time.time()

# Executar o Grid Search
grid_search.fit(X_train, y_train)

end_time = time.time()
print(f"Grid Search concluído em {end_time - start_time:.2f} segundos.")

# Resultados do Melhor Modelo
best_xgb = grid_search.best_estimator_
best_score = grid_search.best_score_
best_params = grid_search.best_params_

print("\nResultados do Grid Search")
print(f"Melhor Score (Recall na Validação Cruzada): {best_score:.4f}")
print("Melhores Parâmetros Encontrados:")
for param, value in best_params.items():
    print(f"  {param}: {value}")

Iniciando Grid Search com 162 combinações...
Fitting 5 folds for each of 162 candidates, totalling 810 fits
Grid Search concluído em 8.44 segundos.

Resultados do Grid Search
Melhor Score (Recall na Validação Cruzada): 0.9313
Melhores Parâmetros Encontrados:
  learning_rate: 0.01
  max_depth: 3
  reg_alpha: 0.1
  reg_lambda: 5
  scale_pos_weight: 3.75


## STEP 8.0.2 Final Evaluation and Final Test.

In [ ]:
# AVALIAÇÃO DO CONJUNTO DE VALIDAÇÃO (X_val)
# Reutilizamos o modelo best_xgb encontrado no Grid Search
y_pred_best_val = best_xgb.predict(X_val)
y_scores_best_val = best_xgb.predict_proba(X_val)[:, 1]

# Cálculo das Métricas de Validaçao
report_val = classification_report(y_val, y_pred_best_val, output_dict=True)
recall_val = report_val["1"]["recall"]
auc_roc_val = roc_auc_score(y_val, y_scores_best_val)
f1_val = report_val["1"]["f1-score"]

print("[A] Avaliação no Conjunto de VALIDAÇÃO")
print(f"Recall (Cl. 1) na Validação: {recall_val:.4f}")
print(f"AUC-ROC na Validação: {auc_roc_val:.4f}")
print(f"F1-Score (Cl. 1) na Validação: {f1_val:.4f}")

[A] Avaliação no Conjunto de VALIDAÇÃO
Recall (Cl. 1) na Validação: 0.9815
AUC-ROC na Validação: 0.8698
F1-Score (Cl. 1) na Validação: 0.6709


In [52]:
# AVALIAÇÃO DO CONJUNTO DE TESTE (X_test)
y_pred_final = best_xgb.predict(X_test)
y_scores_final = best_xgb.predict_proba(X_test)[:, 1]

# Cálculo das Métricas Finais
report_test = classification_report(y_test, y_pred_final, output_dict=True)
recall_test = report_test["1"]["recall"]
auc_roc_test = roc_auc_score(y_test, y_scores_final)
f1_test = report_test["1"]["f1-score"]
accuracy_test = accuracy_score(y_test, y_pred_final)

print("\nAVALIAÇÃO FINAL NO CONJUNTO DE TESTE")
print("Relatório de Classificação Final:")
print(classification_report(y_test, y_pred_final))
print("-" * 100)
print(f"Recall (Cl. 1) FINAL: {recall_test:.4f} (Métrica Crítica)")
print(f"F1-Score (Cl. 1) FINAL: {f1_test:.4f}")
print(f"AUC-ROC FINAL: {auc_roc_test:.4f}")

# Matriz de Confusão Final
cm_final = confusion_matrix(y_test, y_pred_final)
cm_labels = ["Não Diabético (0)", "Diabético (1)"]

fig_cm_final = ff.create_annotated_heatmap(
    z=cm_final,
    x=cm_labels,
    y=cm_labels,
    colorscale="blues",
    annotation_text=cm_final,
    hoverinfo="z",
)

fig_cm_final.update_layout(
    title_text=f"<b>Matriz de Confusão Final - Recall {recall_test:.4f}</b>",
    xaxis=dict(title="Rótulos Preditos"),
    yaxis=dict(title="Rótulos Verdadeiros", autorange="reversed"),
)

fig_cm_final.show()


AVALIAÇÃO FINAL NO CONJUNTO DE TESTE
Relatório de Classificação Final:
              precision    recall  f1-score   support

           0       0.94      0.49      0.64       100
           1       0.50      0.94      0.65        54

    accuracy                           0.65       154
   macro avg       0.72      0.72      0.65       154
weighted avg       0.79      0.65      0.65       154

----------------------------------------------------------------------------------------------------
Recall (Cl. 1) FINAL: 0.9444 (Métrica Crítica)
F1-Score (Cl. 1) FINAL: 0.6538
AUC-ROC FINAL: 0.8116


## STEP 8.0.3 Final considerations of the tests

> **Sucesso do Hyperparameter Fine-Tuning** 
- A otimização através do GridSearchCV, utilizando o Recall da Classe 1 (recall) como métrica de scoring, foi decisiva para resolver o problema de viés e baixo Recall identificado nas etapas iniciais (0.59).
- O modelo otimizado **(best_xgb) atingiu um Recall de 0.9815 na Validação Cruzada**, indicando o potencial máximo do modelo.
- O ajuste fino encontrou um alto peso para a classe positiva (scale_pos_weight: 3.75), e parâmetros de regularização (max_depth: 3, reg_lambda: 5) que confirmaram a escolha do modelo.

> **Performance Final Não-Enviesada (Conjunto de Teste)** 
- O desempenho final do XGBoost Classifier no conjunto de Teste confirma o sucesso da otimização: 
    - Recall (Cl. 1) Final: 0.9444 
    - AUC-ROC Final: 0.8116
    - Falsos Negativos (FN): Apenas 3} casos em 54.

> **O modelo final demonstra um Trade-off Ótimo para o cenário de Triagem Clínica**:
- Ganho: O Recall de 0.9444 significa que 94.4% dos pacientes diabéticos reais foram corretamente identificados. 
- O erro de Falso Negativo (3 casos) é o menor possível, minimizando o risco clínico.
- Custo: O custo é a Precision de 0.50 (e Accuracy de 0.65). Isso gera Falsos Positivos (51 casos de não-diabéticos erroneamente marcados).

> **Conclusão:** 
- É preferível que a ferramenta de triagem seja altamente sensível (alto Recall), marcando mais pacientes saudáveis para reavaliação (Falsos Positivos) do que perder um diagnóstico real de diabetes (Falsos Negativos). 
- Esses resultados refletem o compromisso ético e prático da modelagem de risco em saúde, **aceitar um maior número de encaminhamentos desnecessários (falsos positivos)**, para garantir que pouquíssimos casos de diabetes não sejam identificados, minimizando riscos para o paciente.


# STEP 9.0 - FINAL PROJECT COMPLETION AND PREPARATION FOR IMPLEMENTATION

## STEP 9.1 Project Summary and Final Model Performance

> **Breve Sumário do Pipeline**
- O projeto seguiu a metodologia CRISP-DM, focando na otimização da variável alvo **(outcome)** que apresentava um desbalanceamento moderado 65% e 35% (STEP 3.1.1).

- A fase de Feature Selection resultou em um conjunto otimizado de 8 features clínicas, as flags foram removidas por baixa importância (STEP 3.1.2 e 4.0).

- O modelo final foi o XGBoost Classifier, selecionado e ajustado através de Grid Search com foco na maximização do Recall da Classe 1 (Diabético) (STEP 8).

> **Conclusão de valor**
- O objetivo principal do projeto de triagem clinica era **minimizar** o risco de Falsos Positivos (pacientes diabéticos não detectados)

    - **Antes da otimização:** O modelo baseline falhava 22 diagnósticos em 54 casos.
    - **Após ajuste fino:** O modelo final reduz esse erro critico para apenas **3 Falsos Negativos**

- O modelo final optou por sacrificar a Precision e a Accuracy resultando em mais **Falsos Positivos** mas essa escolha e clinicamente justificada, é preferível que a ferramenta de triagem chame 51 pacientes saudáveis para uma reavaliação do que perder 22 casos de diabetes. 

## STEP 9.2  Project Achievements and Key Insights

> **1. Descobertas e Tratamento de Dados (Data Preparation)**

- **Entendimento de zeros clínicos:** A análise exploratória (EDA) revelou que as variáveis Glucose, BloodPressure, SkinThickness, Insulin e BMI continham valores zero, que são clinicamente impossíveis ou improváveis. 

- **Estratégia de tratamento:** A solução adotada foi a imputação pela mediana para mitigar o efeito de outliers na distribuição.

- **Seleção de features (Remoção de flags):** Embora a criação das flags binárias para indicar a presença de NaNs tenha sido uma etapa inicial de exploração, a análise de importância de features confirmou que elas (flags) tinham baixo poder preditivo.


> **2. Hierarquia e Importância dos Preditors**
- O projeto quantificou a influência de cada variável na previsão de diabetes:
- A variável Glucose é o preditor mais forte para o outcome, seguida por BMI (Índice de Massa Corporal) e Age.

> **3. Validação da Estratégia de Otimização**
- Identificação do Problema: Modelos iniciais (Baseline) falharam em atingir a segurança clínica devido ao baixo Recall (0.59) no conjunto de Teste.
- Estratégia Corretiva: A decisão de focar o Grid Search na maximização do Recall da Classe 1, utilizando o parâmetro scale_pos_weight, foi o fator de sucesso. 
- Esta estratégia transformou o modelo, elevando o Recall de 0.59 para 0.94 no Teste.


## STEP 9.3 Next Steps and Deployment Readiness

> **1. Modelo Final para Deployment**
- Modelo Selecionado: XGBoost Classifier
- Ajuste Fino Aplicado: scale_pos_weight: 3.75 (e outros hyperparâmetros de regularização)
- Próxima Ação Imediata (Treinamento Final): O modelo deve ser retreinado uma última vez no conjunto completo de Treino + Validação (o conjunto X_rem), utilizando os parâmetros ideais encontrados no Grid Search. 

> **2. Requisitos de Deployment (Implantação)**
- Para que o modelo funcione em um ambiente de produção, dois artefatos principais precisam ser salvos:
    - **Objeto 1: O Modelo Treinado:** O objeto best_xgb deve ser serializado (salvo em disco usando joblib ou pickle).
    - **Objeto 2: O Scaler:** O objeto StandardScaler (que foi ajustado apenas no X_train no STEP 6.2) também deve ser salvo, pois os novos dados de entrada (produção) devem ser padronizados usando as mesmas médias e desvios padrão aprendidos durante o treinamento.

- O pipeline de inferência em produção será: 
>>Dados Brutos -> Carregar Scaler -> Padronizar -> Carregar Modelo -> Previsão

> **3. Plano de Manutenção e Monitoramento**
- **Métrica Chave em Produção:**
    - O Recall da Classe 1 deve ser a métrica prioritária de monitoramento. Se o Recall em dados reais de produção cair significativamente abaixo de 0.90, um alerta deve ser acionado.
- **Drift de Dados:** 
    - A distribuição das features (como glucose e bmi) deve ser monitorada para data drift (mudanças no comportamento dos dados ao longo do tempo).
- **Retreinamento:** O modelo deve ser retreinado periodicamente (a cada 6 ou 12 meses) com novos dados para garantir que ele se adapte a quaisquer mudanças nos padrões de saúde da população.

> **4. Limitações do Modelo**
- É fundamental reconhecer as restrições inerentes aos dados:
- **População Específica:** O modelo foi treinado exclusivamente em mulheres Pima Indian. Não pode ser generalizado para outras etnias ou populações (homens, outras idades, etc.) sem retreinamento e validação específicos.
- **Trade-off de Precision:** O alto Recall (0.94) foi alcançado ao custo de uma baixa Precision (0.50), o que significa que cerca de metade dos pacientes classificados como diabéticos serão Falsos Positivos. 

# STEP 10 - DEPLOYMENT

## STEP 10.1 Path Folder

In [53]:
DEPLOYMENT_PATH = BASE_DIR / "deployment_artifacts"
os.makedirs(DEPLOYMENT_PATH, exist_ok=True)

## STEP 10.2 Auxiliary functions

In [ ]:
# Colunas que tiveram 0 imputados pela mediana
IMPUTE_COLS = ["glucose", "blood_pressure", "skin_thickness", "insulin", "bmi"]

def train_and_save_artifacts(df_clean, target_col, best_params):
    """
    Treina o modelo XGBoost final e o StandardScaler no conjunto completo de
    treinamento/validação (X_rem) e salva ambos para produção.

    Args:
        df_clean (pd.DataFrame): DF não padronizado, após seleção de features (as 8 features + target).
        target_col (str): Nome da coluna target ('outcome').
        best_params (dict): Hyperparâmetros encontrados pelo Grid Search (STEP 8.0.1).
    """
    print("1. Preparando Dados para Retreinamento Final")

    # Recriando o conjunto X_rem (Treino + Validação)
    X_rem = df_clean.drop(columns=[target_col]).copy()
    y_rem = df_clean[target_col].copy()
    NUM_FEATURES = X_rem.columns.tolist()

    # Calculamos as medianas nas colunas que tiveram 0s
    imputation_values = df_clean[IMPUTE_COLS].replace(0, np.nan).median().to_dict()
    joblib.dump(
        imputation_values,
        os.path.join(DEPLOYMENT_PATH, "diabetes_imputer_median.joblib"),
    )
    print("Medianas de imputação salvas.")

    # Aplicar a imputação no DF que será treinado (X_rem)
    for col, median in imputation_values.items():
        X_rem[col] = X_rem[col].replace(0, median)

    # 2. FIT do SCALER no X_rem completo (Treino + Validação)
    scaler = StandardScaler()
    X_rem_scaled = scaler.fit_transform(X_rem[NUM_FEATURES])
    X_rem[NUM_FEATURES] = X_rem_scaled

    # 3. Treinar o Modelo Final com os Parâmetros Ótimos
    print("2. Treinamento do XGBoost Final")

    model_final = XGBClassifier(
        objective="binary:logistic",
        eval_metric="logloss",
        random_state=42,
        **best_params,  # Desempacota os parâmetros (learning_rate, max_depth, scale_pos_weight, etc.)
    )
    model_final.fit(X_rem, y_rem)

    # 4. Serializar Artefatos
    joblib.dump(model_final, os.path.join(DEPLOYMENT_PATH, "diabetes_xgb_model.joblib"))
    joblib.dump(scaler, os.path.join(DEPLOYMENT_PATH, "diabetes_scaler.joblib"))

    print("\nTreinamento final concluído. Modelo e Scaler salvos para Deployment.")
    return model_final, scaler


def preprocess_for_api(input_df, deployment_path=DEPLOYMENT_PATH):
    """
    Aplica a lógica de Feature Engineering: Imputa 0s com as medianas salvas.
    """
    IMPUTE_COLS = ["glucose", "blood_pressure", "skin_thickness", "insulin", "bmi"]

    try:
        imputation_values = joblib.load(
            os.path.join(deployment_path, "diabetes_imputer_median.joblib")
        )
    except FileNotFoundError:
        raise Exception(
            "Artefato de imputação (medianas) não encontrado. Treine o modelo primeiro (train_and_save_artifacts)."
        )

    processed_df = input_df.copy()

    # Aplicar a regra de imputação, 0 são substituídos pelas medianas
    for col in IMPUTE_COLS:
        if col in processed_df.columns:
            median = imputation_values.get(col)
            # Substituir 0 pela mediana aprendida (ou usar NaN se for o caso)
            processed_df[col] = processed_df[col].replace(0, median)

    return processed_df


def load_and_predict(new_patient_data, feature_names, deployment_path=DEPLOYMENT_PATH):
    """
    Carrega modelo/scaler e faz a previsão.

    Args:
        new_patient_data (list/array): Lista dos 8 valores de features (NÃO PADRONIZADOS).
        feature_names (list): Nomes das 8 features na ordem correta.
    """
    try:
        # 1. Carregar Artefatos
        loaded_scaler = joblib.load(
            os.path.join(deployment_path, "diabetes_scaler.joblib")
        )
        loaded_model = joblib.load(
            os.path.join(deployment_path, "diabetes_xgb_model.joblib")
        )

        # 2. Preparar Dados de Entrada
        input_df = pd.DataFrame([new_patient_data], columns=feature_names)

        # 3. FEATURE ENGINEERING (IMPUTAÇÃO NaNs) ---
        input_df_imputed = preprocess_for_api(input_df, deployment_path)

        # 4. PADRONIZAÇÃO (Usa o scaler carregado)
        input_scaled = loaded_scaler.transform(input_df_imputed)

        # 5. PREVISÃO
        prediction = loaded_model.predict(input_scaled)
        proba = loaded_model.predict_proba(input_scaled)[:, 1]

        result = {
            "features_input": input_df.to_dict("records")[0],
            "prediction": int(prediction[0]),
            "probability_diabetic": float(proba[0]),
            "diagnosis": (
                "Diabético (Alto Risco)"
                if prediction[0] == 1
                else "Não Diabético (Risco Normal)"
            ),
        }
        return result

    except FileNotFoundError:
        return {
            "error": "Arquivos de modelo ou scaler não encontrados. Execute o train_and_save_artifacts primeiro."
        }

## STEP 10.3 We took the best parameters from STEP 8.0.1

In [60]:
best_params_found = {
    "learning_rate": 0.01,
    "max_depth": 3,
    "reg_alpha": 0.1,
    "reg_lambda": 5,
    "scale_pos_weight": 3.75,
}

# Assumindo df4 (o DF limpo de 9 colunas) pode ser df6 ou o DF mais limpo.
df_for_final_train = df4.copy()
model_final, scaler_final = train_and_save_artifacts(df_for_final_train, 'outcome', best_params_found)

1. Preparando Dados para Retreinamento Final
Medianas de imputação salvas.
2. Treinamento do XGBoost Final

Treinamento final concluído. Modelo e Scaler salvos para Deployment.


## STEP 10.4 Run the inference simulation

In [ ]:
# Exemplo de um paciente de alto risco (Glucose alta, BMI alto)
# A ordem das 8 features deve ser: pregnancies, glucose, blood_pressure, skin_thickness, insulin, bmi, diabetes_pedigree_function, age
patient_example = [3, 160.0, 72.0, 30.0, 150.0, 35.0, 0.45, 35.0]

# Assumindo que são utilizadas as 8 features finais (X_train.columns.tolist())
feature_list = [
    "pregnancies",
    "glucose",
    "blood_pressure",
    "skin_thickness",
    "insulin",
    "bmi",
    "diabetes_pedigree_function",
    "age",
]

final_result = load_and_predict(patient_example, feature_list)
print("\nSimulação de Resposta da API")
print(json.dumps(final_result, indent=4))


Simulação de Resposta da API
{
    "features_input": {
        "pregnancies": 3,
        "glucose": 160.0,
        "blood_pressure": 72.0,
        "skin_thickness": 30.0,
        "insulin": 150.0,
        "bmi": 35.0,
        "diabetes_pedigree_function": 0.45,
        "age": 35.0
    },
    "prediction": 1,
    "probability_diabetic": 0.8312564492225647,
    "diagnosis": "Diab\u00e9tico (Alto Risco)"
}


In [ ]:
# Assumindo que são utilizadas as 8 features finais (X_train.columns.tolist())
feature_list = [
    "pregnancies",
    "glucose",
    "blood_pressure",
    "skin_thickness",
    "insulin",
    "bmi",
    "diabetes_pedigree_function",
    "age",
]

# 10 Pacientes com Perfis Variados
# (Ordem das features: pregnancies, glucose, bp, skin, insulin, bmi, dpf, age)
patient_scenarios = [
    # 1. Diabético Óbvio (HIGH RISK - GLICOSE MUITO ALTA)
    [2, 185.0, 80.0, 30.0, 0.0, 42.0, 0.55, 45.0],
    # 2. Não Diabético Óbvio (LOW RISK - GLICOSE BAIXA)
    [1, 95.0, 68.0, 22.0, 0.0, 25.0, 0.2, 25.0],
    # 3. Risco Limítrofe (GLICOSE MODERADA/BMI ALTO) - Testando a precisão do modelo
    [4, 125.0, 78.0, 35.0, 0.0, 34.5, 0.3, 38.0],
    # 4. Alto Risco (Idade/Histórico Familiar Alto) - Testando DPF/Age
    [6, 140.0, 70.0, 0.0, 0.0, 30.0, 1.2, 55.0],
    # 5. Alto Risco (Insulina Alta) - Testando a feature insulin
    [0, 155.0, 70.0, 30.0, 350.0, 32.0, 0.4, 30.0],
    # 6. Não Diabético Jovem (Perfil Saudável)
    [0, 105.0, 60.0, 25.0, 50.0, 22.0, 0.1, 21.0],
    # 7. Caso FN Potencial (Perfil de risco, mas não é diabético) - Testando a tolerância
    [5, 130.0, 82.0, 0.0, 0.0, 36.0, 0.6, 42.0],
    # 8. Diabético Confirmado (Simples)
    [3, 160.0, 72.0, 30.0, 150.0, 33.0, 0.4, 35.0],
    # 9. Risco Extremo (Combinação de muitos fatores)
    [8, 175.0, 100.0, 45.0, 200.0, 45.0, 0.8, 60.0],
    # 10. Limítrofe Baixo (Glicose OK, mas Idade/BMI na média)
    [2, 115.0, 70.0, 0.0, 0.0, 28.0, 0.35, 30.0],
]

results_list = []
print("\nExecutando Simulação de Inferência para 10 Pacientes")
print("-" * 100)

for i, data in enumerate(patient_scenarios):
    result = load_and_predict(data, feature_list)

    diag = result["diagnosis"]
    proba = result["probability_diabetic"]

    print(f"Paciente {i+1}: Glicose={data[1]}, BMI={data[5]}, Idade={data[7]}")
    print(f"  -> Resultado: {diag} | Probabilidade: {proba:.4f}\n")

    results_list.append(result)


Executando Simulação de Inferência para 10 Pacientes
----------------------------------------------------------------------------------------------------
Paciente 1: Glicose=185.0, BMI=42.0, Idade=45.0
  -> Resultado: Diabético (Alto Risco) | Probabilidade: 0.8313

Paciente 2: Glicose=95.0, BMI=25.0, Idade=25.0
  -> Resultado: Não Diabético (Risco Normal) | Probabilidade: 0.3038

Paciente 3: Glicose=125.0, BMI=34.5, Idade=38.0
  -> Resultado: Diabético (Alto Risco) | Probabilidade: 0.7412

Paciente 4: Glicose=140.0, BMI=30.0, Idade=55.0
  -> Resultado: Diabético (Alto Risco) | Probabilidade: 0.7843

Paciente 5: Glicose=155.0, BMI=32.0, Idade=30.0
  -> Resultado: Diabético (Alto Risco) | Probabilidade: 0.8221

Paciente 6: Glicose=105.0, BMI=22.0, Idade=21.0
  -> Resultado: Não Diabético (Risco Normal) | Probabilidade: 0.3111

Paciente 7: Glicose=130.0, BMI=36.0, Idade=42.0
  -> Resultado: Diabético (Alto Risco) | Probabilidade: 0.7843

Paciente 8: Glicose=160.0, BMI=33.0, Idade=35.0
  